# Opening

In [ ]:
import cv2, numpy as np, os


def detect_gaps(img_path):
    img = cv2.imread(img_path, 0)
    _, bin_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    skel = np.zeros(bin_img.shape, np.uint8)
    temp = np.zeros(bin_img.shape, np.uint8)
    eroded = np.zeros(bin_img.shape, np.uint8)
    elem = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    done = False
    while not done:
        eroded = cv2.erode(bin_img, elem)
        temp = cv2.dilate(eroded, elem)
        temp = cv2.subtract(bin_img, temp)
        cv2.bitwise_or(skel, temp, skel)
        bin_img = eroded.copy()
        if cv2.countNonZero(bin_img) == 0:
            done = True

    gap_points = []
    rows, cols = skel.shape
    for r in range(1, rows - 1):
        for c in range(1, cols - 1):
            if skel[r, c] == 255:
                area = skel[r - 1 : r + 2, c - 1 : c + 2]
                if cv2.countNonZero(area) == 2:
                    gap_points.append((c, r))

    out = cv2.imread(img_path)
    for i in range(len(gap_points) - 1):
        x1, y1 = gap_points[i]
        x2, y2 = gap_points[i + 1]
        dist = np.hypot(x2 - x1, y2 - y1)
        if dist < 50:
            mx = (x1 + x2) // 2
            my = (y1 + y2) // 2
            cv2.circle(out, (mx, my), 10, (0, 0, 255), 3)

    cv2.imshow("Hasil Gap Detection", out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


detect_gaps(r"C:\xampp\htdocs\VISUALAI\website-django\inspection\static\images\test\disconnected.jpg")

In [ ]:
import cv2, numpy as np

cap = cv2.VideoCapture(1)


def resize(img, target_w):
    h, w = img.shape[:2]
    return cv2.resize(img, (target_w, int(h * target_w / w)))


def binary_frame_by_dominant_colors(img):
    data = img.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret, labels, centers = cv2.kmeans(data, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    brightness = []
    for center in centers:
        b, g, r = center
        brightness.append(0.114 * b + 0.587 * g + 0.299 * r)
    idx_bright = np.argmax(brightness)
    binary = np.where(labels.flatten() == idx_bright, 255, 0).astype(np.uint8)
    binary = binary.reshape(img.shape[:2])
    return cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)


w_target = 360

while True:
    ret, frame = cap.read()
    if not ret:
        break
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    binary = binary_frame_by_dominant_colors(frame)
    f1 = resize(frame, w_target)
    f2 = resize(hsv, w_target)
    f3 = resize(gray, w_target)
    f4 = resize(binary, w_target)
    top = np.hstack([f1, f2])
    bottom = np.hstack([f3, f4])
    grid = np.vstack([top, bottom])
    cv2.imshow("Grid", grid)
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

cap.release()
cv2.destroyAllWindows()

# Path Detection

In [ ]:
import cv2
import numpy as np


def detect_c_breaks_bbox(img_path, gap_threshold=20):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Jika huruf lebih gelap, pakai THRESH_BINARY, sesuaikan
    _, bin_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Cari connected components
    # stats: [x, y, width, height, area]
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_img, connectivity=8)

    # label 0 = background, sisanya objek
    bboxes = []
    for label_id in range(1, num_labels):
        x, y, w, h, area = stats[label_id]
        if area > 10:
            bboxes.append((x, y, x + w, y + h))  # (left, top, right, bottom)

    if len(bboxes) <= 1:
        print("Tidak terdeteksi lebih dari 1 huruf C, dianggap PASS.")
        cv2.imshow("Result", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return

    # Urutkan berdasarkan top (y)
    bboxes.sort(key=lambda b: b[1])

    # Periksa gap di antara bounding box berurutan
    # bottom dari box atas = b[3], top dari box bawah = b'[1]
    putus_detected = False
    for i in range(len(bboxes) - 1):
        leftA, topA, rightA, bottomA = bboxes[i]
        leftB, topB, rightB, bottomB = bboxes[i + 1]

        distance = topB - bottomA
        if distance > gap_threshold:
            putus_detected = True
            # Tentukan titik tengah gap
            # X ambil rata-rata tengah bounding box A dan B
            midX_A = (leftA + rightA) // 2
            midX_B = (leftB + rightB) // 2
            midX_gap = (midX_A + midX_B) // 2

            # Y di tengah jarak antara bottomA dan topB
            midY_gap = bottomA + distance // 2

            cv2.circle(img, (midX_gap, midY_gap), 10, (0, 0, 255), 3)

    if putus_detected:
        print("WARNING: Ada huruf C terputus. Lingkaran merah menunjukkan lokasi gap.")
    else:
        print("PASS: Tidak ada gap berarti di antara huruf C.")
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    
    cv2.destroyAllWindows()


detect_c_breaks_bbox(r"C:\xampp\htdocs\VISUALAI\website-django\inspection\static\images\test\disconnected.jpg", gap_threshold=30)

In [ ]:
import cv2, numpy as np, cvzone


def detect_c_breaks_bbox_red(frame, gap_threshold=20):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 70, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 70])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    bboxes = []
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > 10:
            bboxes.append((x, y, x + w, y + h))
    if len(bboxes) <= 1:
        return frame, False

    bboxes.sort(key=lambda b: b[1])
    putus = False
    for i in range(len(bboxes) - 1):
        lA, tA, rA, bA = bboxes[i]
        lB, tB, rB, bB = bboxes[i + 1]
        distance = tB - bA
        if distance > gap_threshold:
            putus = True
            mx_gap = ((lA + rA) // 2 + (lB + rB) // 2) // 2
            my_gap = bA + distance // 2
            cv2.circle(frame, (mx_gap, my_gap), 10, (0, 0, 255), 3)
    return frame, putus


def main():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("No camera found.")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        result_frame, is_broken = detect_c_breaks_bbox_red(frame, gap_threshold=30)
        if is_broken:
            # cv2.putText(result_frame, "REJECT: Ada Jahitan Terlewat", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cvzone.putTextRect(result_frame, "Reject: Ada Jahitan Terlewat", (20, 40), 1, 2, offset=5, border=2, colorR=(0, 255, 255), colorT=(0, 0, 0), colorB=(255, 255, 255))
        else:
            # cv2.putText(result_frame, "GOOD: Tidak Ada Jahitan Terlewat", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cvzone.putTextRect(result_frame, "Good: Tidak Ada Jahitan Terlewat", (20, 40), 1, 2, offset=5, border=2, colorR=(0, 255, 0), colorT=(0, 0, 0), colorB=(255, 255, 255))
        cv2.imshow("C Break Detection (Red)", result_frame)
        if cv2.waitKey(1) & 0xFF == ord("n"):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

# HSV

In [ ]:
import cv2, numpy as np

cap = cv2.VideoCapture(1)


def resize(img, target_w):
    h, w = img.shape[:2]
    return cv2.resize(img, (target_w, int(h * target_w / w)))


def get_red_mask(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    return red_mask


def enforce_dominant_line(img, red_mask):
    edges = cv2.Canny(red_mask, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)
    mask = np.ones_like(red_mask, dtype=np.uint8) * 255
    if lines is not None:
        longest_line = None
        max_length = 0
        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = np.hypot(x2 - x1, y2 - y1)
            if length > max_length:
                max_length = length
                longest_line = (x1, y1, x2, y2)
        if longest_line is not None:
            x1, y1, x2, y2 = longest_line
            cv2.line(mask, (x1, y1), (x2, y2), 0, 5)
    refined = np.where(mask[..., None] == 0, 0, img)
    return refined


w_target = 360

while True:
    ret, frame = cap.read()
    if not ret:
        break

    red_mask = get_red_mask(frame)
    refined = enforce_dominant_line(frame.copy(), red_mask)
    f1 = resize(frame, w_target)  # Citra asli
    f2 = resize(cv2.cvtColor(red_mask, cv2.COLOR_GRAY2BGR), w_target)  # Mask merah
    f3 = resize(refined, w_target)  # Citra dengan garis dominan
    grid = np.hstack([f1, f2, f3])
    cv2.imshow("Deteksi Garis Berdasarkan Warna", grid)
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

cap.release()
cv2.destroyAllWindows()

# Pytorch Computation

In [ ]:
import cv2, numpy as np, torch, kornia

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


def resize(img, target_w):
    h, w = img.shape[:2]
    return cv2.resize(img, (target_w, int(h * target_w / w)))

def get_red_mask_tensor_white_fabric(frame):
    img_tensor = torch.from_numpy(frame).to(device, dtype=torch.float32)
    img_tensor = img_tensor[..., [2, 1, 0]]
    R = img_tensor[..., 0]
    G = img_tensor[..., 1]
    B = img_tensor[..., 2]
    red_mask = (R > 100) & ((R - G) > 30) & ((R - B) > 30)
    red_mask = red_mask.to(torch.uint8) * 255
    return red_mask.cpu().numpy()

def enforce_dominant_line(img, red_mask):
    edges = cv2.Canny(red_mask, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)

    mask = np.ones_like(red_mask, dtype=np.uint8) * 255
    if lines is not None:
        longest_line = None
        max_length = 0
        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = np.hypot(x2 - x1, y2 - y1)
            if length > max_length:
                max_length = length
                longest_line = (x1, y1, x2, y2)
        if longest_line is not None:
            x1, y1, x2, y2 = longest_line
            cv2.line(mask, (x1, y1), (x2, y2), 0, 5)
    refined = np.where(mask[..., None] == 0, 0, img)
    return refined


cap = cv2.VideoCapture(1)
w_target = 360

while True:
    ret, frame = cap.read()
    if not ret:
        break

    red_mask = get_red_mask_tensor_white_fabric(frame)
    refined = enforce_dominant_line(frame.copy(), red_mask)

    f1 = resize(frame, w_target)
    f2 = resize(cv2.cvtColor(red_mask, cv2.COLOR_GRAY2BGR), w_target)
    f3 = resize(refined, w_target)

    grid = np.hstack([f1, f2, f3])
    cv2.imshow("Deteksi Garis Berdasarkan Warna", grid)

    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

cap.release()
cv2.destroyAllWindows()

# Pytorch Computation : Add New Custom Color

In [ ]:
import cv2, numpy as np, torch, kornia

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


def resize(img, w):
    h, _ = img.shape[:2]
    return cv2.resize(img, (w, int(h * w / img.shape[1])))


def get_color_mask_tensor(frame, color="red"):
    img = torch.from_numpy(frame).float().to(device)[..., [2, 1, 0]]
    R, G, B = img[..., 0], img[..., 1], img[..., 2]
    if color == "red":
        mask = (R > 100) & ((R - G) > 30) & ((R - B) > 30)
    elif color == "yellow":
        mask = (R > 150) & (G > 150) & (B > 150)
    else:
        mask = torch.zeros_like(R, dtype=torch.bool)
    return (mask * 255).byte().cpu().numpy()


def enforce_dominant_line(img, mask):
    edges = cv2.Canny(mask, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, 30, 10)
    m = np.ones_like(mask) * 255
    if lines is not None:
        mx, ln = 0, None
        for l in lines:
            x1, y1, x2, y2 = l[0]
            dist = np.hypot(x2 - x1, y2 - y1)
            if dist > mx:
                mx, ln = dist, (x1, y1, x2, y2)
        if ln:
            x1, y1, x2, y2 = ln
            cv2.line(m, (x1, y1), (x2, y2), 0, 5)
    return np.where(m[..., None] == 0, 0, img)


cap = cv2.VideoCapture(1)  # ganti ke 1 bila diinginkan
w_target = 360
target_color = "yellow"  # ubah ke 'red' atau 'yellow'

while True:
    ret, frame = cap.read()
    if not ret:
        break
    mask = get_color_mask_tensor(frame, color=target_color)
    refined = enforce_dominant_line(frame.copy(), mask)
    f1 = resize(frame, w_target)
    f2 = resize(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), w_target)
    f3 = resize(refined, w_target)
    cv2.imshow("Color-based Line Detection", np.hstack([f1, f2, f3]))
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

cap.release()
cv2.destroyAllWindows()

# Scikit Learn

In [ ]:
import cv2, numpy as np
from sklearn.cluster import KMeans

img = cv2.imread(r"C:\xampp\htdocs\VISUALAI\website-django\inspection\static\images\test\rapid_test.jpg")
img = cv2.GaussianBlur(img, (5, 5), 0)
h, w = img.shape[:2]
data = img.reshape(-1, 3).astype(np.float32)

k = 2
km = KMeans(n_clusters=k, random_state=42).fit(data)
labels = km.labels_.reshape(h, w)

largest_label = np.bincount(labels.flatten()).argmax()
mask = (labels == largest_label).astype(np.uint8) * 255

cv2.imshow("Dominant-Binary", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2, numpy as np
from sklearn.cluster import KMeans


cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    original_img = frame.copy()
    kernel_slide = 0
    
    blurred_img = cv2.GaussianBlur(frame, (kernel_slide, kernel_slide), 5)
    h, w = blurred_img.shape[:2]
    data = blurred_img.reshape(-1, 3).astype(np.float32)

    k = 2
    km = KMeans(n_clusters=k, random_state=42).fit(data)
    labels = km.labels_.reshape(h, w)

    largest_label = np.bincount(labels.flatten()).argmax()
    mask = (labels == largest_label).astype(np.uint8) * 255

    cv2.namedWindow("ori-blur-bin", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("ori-blur-bin", (1280, 360))

    cv2.imshow("ori-blur-bin", np.hstack([original_img, blurred_img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)]))
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break
cap.release()
cv2.destroyAllWindows()